# W_GL_ACCOUNT_D ETL Process
### SCD Type 1 Dimension - Staging to Dimension with Segment Hierarchy

In [ ]:
%sql
-- 1) PARAMETERS
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID, 2625278 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLACCOUNTDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING WITH CHANGE DETECTION (Simplified for core logic)
CREATE OR REPLACE TEMP VIEW stg_account_with_action AS
SELECT
  ds.*,
  CASE
    WHEN t.ROW_WID IS NOT NULL
      AND (COALESCE(t.CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(ds.CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')))
      AND (COALESCE(t.AUX1_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(ds.AUX1_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')))
    THEN 'N'
    WHEN t.ROW_WID IS NOT NULL THEN 'U'
    ELSE 'I'
  END AS IND_UPDATE
FROM workspace.oracle_edw.w_gl_account_ds ds
LEFT JOIN workspace.oracle_edw.w_gl_account_d t
  ON ds.DATASOURCE_NUM_ID = t.DATASOURCE_NUM_ID
  AND ds.INTEGRATION_ID = t.INTEGRATION_ID;

In [ ]:
%sql
-- 3) MERGE INTO TARGET
MERGE INTO workspace.oracle_edw.w_gl_account_d AS tgt
USING (
  SELECT s.*, p.ETL_PROC_WID, current_timestamp() AS CURRENT_TS
  FROM stg_account_with_action s
  CROSS JOIN etl_params p
  WHERE s.IND_UPDATE IN ('I', 'U')
) src
ON tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
WHEN MATCHED AND src.IND_UPDATE = 'U' THEN
  UPDATE SET
    tgt.CONCATENATED_SEGMENT_VALUE = src.CONCATENATED_SEGMENT_VALUE,
    tgt.CHART_OF_ACCOUNTS = src.CHART_OF_ACCOUNTS,
    tgt.BALANCING_SEGMENT_NUM = src.BALANCING_SEGMENT_NUM,
    tgt.COST_CENTER_NUM = src.COST_CENTER_NUM,
    tgt.NATURAL_ACCOUNT_NUM = src.NATURAL_ACCOUNT_NUM,
    tgt.ACTIVE_FLG = src.ACTIVE_FLG,
    tgt.W_UPDATE_DT = src.CURRENT_TS,
    tgt.ETL_PROC_WID = src.ETL_PROC_WID
WHEN NOT MATCHED AND src.IND_UPDATE = 'I' THEN
  INSERT (
    CONCATENATED_SEGMENT_VALUE, CHART_OF_ACCOUNTS,
    BALANCING_SEGMENT_NUM, COST_CENTER_NUM, NATURAL_ACCOUNT_NUM,
    ACTIVE_FLG, DATASOURCE_NUM_ID, INTEGRATION_ID,
    W_INSERT_DT, W_UPDATE_DT, ETL_PROC_WID
  )
  VALUES (
    src.CONCATENATED_SEGMENT_VALUE, src.CHART_OF_ACCOUNTS,
    src.BALANCING_SEGMENT_NUM, src.COST_CENTER_NUM, src.NATURAL_ACCOUNT_NUM,
    src.ACTIVE_FLG, src.DATASOURCE_NUM_ID, src.INTEGRATION_ID,
    src.CURRENT_TS, src.CURRENT_TS, src.ETL_PROC_WID
  );

In [ ]:
%sql
-- 4) UPDATE CONTROL (Standard pattern)
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLACCOUNTDIMENSION' AS package_name, 'W_GL_ACCOUNT_D' AS target_table_name,
    p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id, current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.etl_load_date = src.etl_load_date
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name,
  src.etl_usage_code, src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
DROP VIEW IF EXISTS stg_account_with_action;
DROP VIEW IF EXISTS etl_params;